In [1]:
import import_ipynb
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb
from simple_rnnlm import SimpleRnnlm

importing Jupyter notebook from ../common/optimizer.ipynb
importing Jupyter notebook from ../dataset/ptb.ipynb
importing Jupyter notebook from simple_rnnlm.ipynb
importing Jupyter notebook from ../common/time_layers.ipynb
importing Jupyter notebook from ../common/layers.ipynb
importing Jupyter notebook from ../common/functions.ipynb


In [2]:
# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5
lr = 0.1
max_epoch = 100

In [3]:
# 학습 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1] #입력
ts = corpus[1:]
data_size = len(xs)
print('말뭉치 크기 : {}, 어휘 수 : {}'.format(corpus_size, vocab_size))

말뭉치 크기 : 1000, 어휘 수 : 418


In [4]:
# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

In [5]:
# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

In [6]:
# 각 미니배치에서 샘플을 읽기 시작할 위치
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)]
offsets

[0, 99, 198, 297, 396, 495, 594, 693, 792, 891]

In [7]:
for epoch in range(max_epoch):
    for iter in range(max_iters):
        # 미니배치 획득
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1
            
            print(batch_x)
            # 기울기를 구하여 매개변수 생성
            loss = model.forward(batch_x, batch_t)
            model.backward()
            optimizer.update(model.params, model.grads)
            total_loss += loss
            loss_count += 1
            
        #에폭마다 퍼블렉서티 평가
        ppl = np.exp(total_loss / loss_count)
        print('| 에폭 {} | 퍼블렉서티 {}'.format(epoch+1, ppl))
        ppl_list.append(float(ppl))
        total_loss, loss_count = 0, 0

[[         0      21990  264279696      21990 -793278928]
 [        42 -793278864      32574 -793278800      32574]
 [        26      32574 -793278672      32574 -793278608]
 [        24 -793278544      32574 -793278480      32574]
 [       208      32574 -793278352      32574 -793278288]
 [        26 -793278224      32574 -793278160      32574]
 [       274      32574 -793278032      32574 -793277968]
 [        88 -793277904      32574 -793277840      32574]
 [        42      32574 -793277712      32574 -793277648]
 [       339 -793277584      32574        208          0]]
[[         0      21990  264279696      21990 -793278928]
 [        42 -793278864      32574 -793278800      32574]
 [        26      32574 -793278672      32574 -793278608]
 [        24 -793278544      32574 -793278480      32574]
 [       208      32574 -793278352      32574 -793278288]
 [        26 -793278224      32574 -793278160      32574]
 [       274      32574 -793278032      32574 -793277968]
 [        88 

IndexError: index 21990 is out of bounds for axis 0 with size 418